# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement
len(twits["data"])



1548010

### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [26]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r"http\S+", "", text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'^\$[a-z]*', ' ', text, flags=re.MULTILINE)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@[a-z0-9]*', ' ', text, flags=re.MULTILINE)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]', ' ', text, flags=re.MULTILINE)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = list([wnl.lemmatize(token, pos='v') for token in tokens])
    
    return [t for t in tokens if t != '' and len(t) != 1]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\surov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [27]:
t1 = preprocess(messages[20])
print(t1, messages[20])

['have', 'current', 'ratio', 'of', 'so', 'it', 'be', 'financially', 'healthy', 'and', 'have', 'no', 'problem', 'in', 'meet', 'its', 'obligations'] $NKE has a Current Ratio of 2.66, so it is financially healthy and has no problem in meeting its obligations. https://www.chartmill.com/analyze.php?utm_source=stocktwits&amp;utm_medium=FA&amp;utm_content=HEALTH&amp;utm_campaign=social_tracking#/NKE?r=fa&amp;key=8832fee1-d06b-46b5-827f-26246a5e29a8


In [28]:
# TODO Implement

tokenized = [preprocess(message) for message in messages]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the |tokens.

In [29]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

counts = Counter()
for tokens in tokenized:
    counts.update(tokens)
    
vocab = sorted(counts, key=counts.get, reverse=True)
bow = {word: ii for ii, word in enumerate(vocab, 0)}

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [40]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = dict(counts.items())

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 20

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 5

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [w for w, _ in counts.most_common(high_cutoff)]

print(K_most_common)

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]

len(filtered_words) 

['be', 'the', 'to', 'for', 'on']


13823

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [41]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = dict((i, w) for w, i in enumerate(filtered_words))
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = dict((v, k) for k, v in vocab.items())
# tokenized with the words not in `filtered_words` removed.


In [42]:
filtered = []
for sentence in tokenized:
    filtered_sentence = [w for w in sentence if w in vocab]
    filtered.append(filtered_sentence)

In [43]:
print(filtered[57])

['eog', 'dvn', 'great', 'summary', 'of', 'jeff', 'gundlach', 'sohn', 'presentation', 'long', 'thesis', 'energy', 'stock']


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [44]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [45]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1959084188689177

Finally let's convert our tokens into integer ids which we can pass to the network.

In [46]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [49]:
print(token_ids[555])
print(sentiments[555])

[407, 78, 76, 439, 89, 502, 503, 126, 336, 48]
4


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [227]:
import torch.nn as nn

class TextClassifier(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(TextClassifier, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """

        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        y = self.fc(lstm_out)
        
        log_probs = F.log_softmax(y)
        log_probs = log_probs.view(batch_size, -1, self.output_size)
        log_probs = log_probs[:, -1]
        
        # return last sigmoid output and hidden state
        return log_probs, hidden
    
    
    def init_hidden(self, batch_size):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.n_layers, batch_size, self.hidden_dim),
                torch.zeros(self.n_layers, batch_size, self.hidden_dim))
    
    


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [258]:
import numpy as np

def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## implement function
    
    features = []
    for review_ints in reviews_ints:
        feature = review_ints
        if len(review_ints) < seq_length:
            feature = np.pad(review_ints, [seq_length - len(review_ints), 0], mode='constant')
        if len(review_ints) > seq_length:
            feature = review_ints[0:seq_length]
        features.append(feature)
    
    return np.array(features)


In [229]:
# seq_length = 40

# features = pad_features(token_ids, seq_length=seq_length)

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [230]:
split_frac = 0.95

split_idx = int(len(features)*0.8)

train_x, remaining_x = np.array(features[:split_idx]), np.array(features[split_idx:])
train_y, remaining_y = np.array(sentiments[:split_idx]), np.array(sentiments[split_idx:])

test_idx = int(len(remaining_x)*0.5)

val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]


In [235]:
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 128

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### Training
It's time to train the neural network!

In [239]:
net = TextClassifier(
    vocab_size=len(vocab) + 1, 
    output_size=5, 
    embedding_dim=512, 
    hidden_dim=1024, 
    n_layers=3, 
    drop_prob=0.3,
)

In [240]:
lr = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [241]:
# training params

epochs = 3
counter = 0
print_every = 20
clip = 5

net.cuda()
net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1
        
        if inputs.size(0) != batch_size:
            continue        

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data.cuda() for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs.long().cuda(), h)
       
        loss = criterion(output, labels.long().cuda())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                
                if inputs.size(0) != batch_size:
                    continue
                
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data.cuda() for each in val_h])

                output, val_h = net(inputs.long().cuda(), val_h)
                val_loss = criterion(output, labels.long().cuda())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

D:\Anaconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/3... Step: 20... Loss: 1.408495... Val Loss: 1.420469
Epoch: 1/3... Step: 40... Loss: 1.377478... Val Loss: 1.324871
Epoch: 1/3... Step: 60... Loss: 1.120266... Val Loss: 1.236577
Epoch: 1/3... Step: 80... Loss: 1.123321... Val Loss: 1.225813
Epoch: 1/3... Step: 100... Loss: 1.183247... Val Loss: 1.178099
Epoch: 1/3... Step: 120... Loss: 1.039853... Val Loss: 1.137612
Epoch: 1/3... Step: 140... Loss: 1.104084... Val Loss: 1.085898
Epoch: 1/3... Step: 160... Loss: 1.080930... Val Loss: 1.064213
Epoch: 1/3... Step: 180... Loss: 1.068344... Val Loss: 1.072467
Epoch: 1/3... Step: 200... Loss: 0.966861... Val Loss: 1.103162
Epoch: 1/3... Step: 220... Loss: 1.061400... Val Loss: 1.023357
Epoch: 1/3... Step: 240... Loss: 1.071752... Val Loss: 1.027660
Epoch: 1/3... Step: 260... Loss: 1.155475... Val Loss: 0.995574
Epoch: 1/3... Step: 280... Loss: 0.959379... Val Loss: 0.985377
Epoch: 1/3... Step: 300... Loss: 0.955211... Val Loss: 0.970278
Epoch: 1/3... Step: 320... Loss: 1.072237...

Epoch: 1/3... Step: 2560... Loss: 0.763695... Val Loss: 0.711259
Epoch: 1/3... Step: 2580... Loss: 0.638049... Val Loss: 0.711049
Epoch: 1/3... Step: 2600... Loss: 0.663536... Val Loss: 0.707910
Epoch: 1/3... Step: 2620... Loss: 0.639386... Val Loss: 0.702825
Epoch: 1/3... Step: 2640... Loss: 0.716825... Val Loss: 0.704462
Epoch: 1/3... Step: 2660... Loss: 0.717182... Val Loss: 0.710145
Epoch: 1/3... Step: 2680... Loss: 0.674172... Val Loss: 0.705072
Epoch: 1/3... Step: 2700... Loss: 0.692797... Val Loss: 0.715499
Epoch: 1/3... Step: 2720... Loss: 0.646143... Val Loss: 0.701450
Epoch: 1/3... Step: 2740... Loss: 0.618910... Val Loss: 0.704678
Epoch: 1/3... Step: 2760... Loss: 0.626202... Val Loss: 0.703796
Epoch: 1/3... Step: 2780... Loss: 0.753574... Val Loss: 0.705428
Epoch: 1/3... Step: 2800... Loss: 0.844469... Val Loss: 0.702747
Epoch: 1/3... Step: 2820... Loss: 0.703125... Val Loss: 0.705815
Epoch: 1/3... Step: 2840... Loss: 0.766880... Val Loss: 0.707098
Epoch: 1/3... Step: 2860.

Epoch: 1/3... Step: 5100... Loss: 0.685475... Val Loss: 0.676968
Epoch: 1/3... Step: 5120... Loss: 0.650060... Val Loss: 0.676500
Epoch: 1/3... Step: 5140... Loss: 0.583289... Val Loss: 0.678960
Epoch: 1/3... Step: 5160... Loss: 0.675449... Val Loss: 0.674445
Epoch: 1/3... Step: 5180... Loss: 0.597429... Val Loss: 0.676833
Epoch: 1/3... Step: 5200... Loss: 0.679035... Val Loss: 0.676890
Epoch: 1/3... Step: 5220... Loss: 0.736520... Val Loss: 0.673509
Epoch: 1/3... Step: 5240... Loss: 0.682854... Val Loss: 0.674498
Epoch: 1/3... Step: 5260... Loss: 0.618153... Val Loss: 0.673963
Epoch: 1/3... Step: 5280... Loss: 0.616496... Val Loss: 0.677718
Epoch: 1/3... Step: 5300... Loss: 0.590143... Val Loss: 0.672706
Epoch: 1/3... Step: 5320... Loss: 0.595730... Val Loss: 0.675800
Epoch: 1/3... Step: 5340... Loss: 0.683004... Val Loss: 0.673041
Epoch: 1/3... Step: 5360... Loss: 0.743446... Val Loss: 0.671560
Epoch: 1/3... Step: 5380... Loss: 0.633994... Val Loss: 0.674623
Epoch: 1/3... Step: 5400.

Epoch: 2/3... Step: 7660... Loss: 0.649050... Val Loss: 0.664513
Epoch: 2/3... Step: 7680... Loss: 0.485664... Val Loss: 0.667233
Epoch: 2/3... Step: 7700... Loss: 0.562289... Val Loss: 0.667753
Epoch: 2/3... Step: 7720... Loss: 0.577129... Val Loss: 0.669630
Epoch: 2/3... Step: 7740... Loss: 0.615036... Val Loss: 0.671250
Epoch: 2/3... Step: 7760... Loss: 0.730297... Val Loss: 0.668649
Epoch: 2/3... Step: 7780... Loss: 0.702256... Val Loss: 0.666707
Epoch: 2/3... Step: 7800... Loss: 0.614878... Val Loss: 0.662844
Epoch: 2/3... Step: 7820... Loss: 0.562064... Val Loss: 0.666695
Epoch: 2/3... Step: 7840... Loss: 0.776009... Val Loss: 0.666856
Epoch: 2/3... Step: 7860... Loss: 0.574400... Val Loss: 0.665709
Epoch: 2/3... Step: 7880... Loss: 0.608509... Val Loss: 0.665334
Epoch: 2/3... Step: 7900... Loss: 0.702410... Val Loss: 0.663979
Epoch: 2/3... Step: 7920... Loss: 0.783828... Val Loss: 0.665720
Epoch: 2/3... Step: 7940... Loss: 0.699250... Val Loss: 0.665209
Epoch: 2/3... Step: 7960.

Epoch: 2/3... Step: 10180... Loss: 0.617953... Val Loss: 0.654576
Epoch: 2/3... Step: 10200... Loss: 0.588351... Val Loss: 0.659576
Epoch: 2/3... Step: 10220... Loss: 0.533683... Val Loss: 0.654269
Epoch: 2/3... Step: 10240... Loss: 0.594823... Val Loss: 0.653840
Epoch: 2/3... Step: 10260... Loss: 0.744546... Val Loss: 0.654660
Epoch: 2/3... Step: 10280... Loss: 0.679215... Val Loss: 0.656352
Epoch: 2/3... Step: 10300... Loss: 0.681086... Val Loss: 0.658150
Epoch: 2/3... Step: 10320... Loss: 0.561296... Val Loss: 0.656814
Epoch: 2/3... Step: 10340... Loss: 0.663897... Val Loss: 0.654080
Epoch: 2/3... Step: 10360... Loss: 0.679183... Val Loss: 0.655227
Epoch: 2/3... Step: 10380... Loss: 0.741814... Val Loss: 0.653992
Epoch: 2/3... Step: 10400... Loss: 0.657551... Val Loss: 0.653660
Epoch: 2/3... Step: 10420... Loss: 0.622758... Val Loss: 0.655052
Epoch: 2/3... Step: 10440... Loss: 0.546074... Val Loss: 0.653769
Epoch: 2/3... Step: 10460... Loss: 0.758680... Val Loss: 0.651989
Epoch: 2/3

Epoch: 2/3... Step: 12680... Loss: 0.623455... Val Loss: 0.648530
Epoch: 2/3... Step: 12700... Loss: 0.667026... Val Loss: 0.648447
Epoch: 2/3... Step: 12720... Loss: 0.692266... Val Loss: 0.648007
Epoch: 2/3... Step: 12740... Loss: 0.602715... Val Loss: 0.647953
Epoch: 2/3... Step: 12760... Loss: 0.491171... Val Loss: 0.645599
Epoch: 2/3... Step: 12780... Loss: 0.564749... Val Loss: 0.646939
Epoch: 2/3... Step: 12800... Loss: 0.765630... Val Loss: 0.651364
Epoch: 2/3... Step: 12820... Loss: 0.528797... Val Loss: 0.647519
Epoch: 2/3... Step: 12840... Loss: 0.652580... Val Loss: 0.648766
Epoch: 2/3... Step: 12860... Loss: 0.637401... Val Loss: 0.645830
Epoch: 2/3... Step: 12880... Loss: 0.742050... Val Loss: 0.648510
Epoch: 2/3... Step: 12900... Loss: 0.546281... Val Loss: 0.647643
Epoch: 3/3... Step: 12940... Loss: 0.551834... Val Loss: 0.649354
Epoch: 3/3... Step: 12960... Loss: 0.511112... Val Loss: 0.654762
Epoch: 3/3... Step: 12980... Loss: 0.387281... Val Loss: 0.659482
Epoch: 3/3

Epoch: 3/3... Step: 15200... Loss: 0.573426... Val Loss: 0.663352
Epoch: 3/3... Step: 15220... Loss: 0.530177... Val Loss: 0.654526
Epoch: 3/3... Step: 15240... Loss: 0.595705... Val Loss: 0.659361
Epoch: 3/3... Step: 15260... Loss: 0.478100... Val Loss: 0.655774
Epoch: 3/3... Step: 15280... Loss: 0.535659... Val Loss: 0.653414
Epoch: 3/3... Step: 15300... Loss: 0.506126... Val Loss: 0.655116
Epoch: 3/3... Step: 15320... Loss: 0.581570... Val Loss: 0.661182
Epoch: 3/3... Step: 15340... Loss: 0.595850... Val Loss: 0.657904
Epoch: 3/3... Step: 15360... Loss: 0.663278... Val Loss: 0.654404
Epoch: 3/3... Step: 15380... Loss: 0.653287... Val Loss: 0.653796
Epoch: 3/3... Step: 15400... Loss: 0.511482... Val Loss: 0.654285
Epoch: 3/3... Step: 15420... Loss: 0.562207... Val Loss: 0.651874
Epoch: 3/3... Step: 15440... Loss: 0.492233... Val Loss: 0.652598
Epoch: 3/3... Step: 15460... Loss: 0.665660... Val Loss: 0.657099
Epoch: 3/3... Step: 15480... Loss: 0.579548... Val Loss: 0.658524
Epoch: 3/3

Epoch: 3/3... Step: 17700... Loss: 0.715067... Val Loss: 0.649947
Epoch: 3/3... Step: 17720... Loss: 0.732966... Val Loss: 0.650656
Epoch: 3/3... Step: 17740... Loss: 0.848637... Val Loss: 0.647761
Epoch: 3/3... Step: 17760... Loss: 0.488574... Val Loss: 0.647575
Epoch: 3/3... Step: 17780... Loss: 0.697920... Val Loss: 0.646637
Epoch: 3/3... Step: 17800... Loss: 0.805694... Val Loss: 0.648389
Epoch: 3/3... Step: 17820... Loss: 0.503881... Val Loss: 0.649885
Epoch: 3/3... Step: 17840... Loss: 0.815714... Val Loss: 0.649900
Epoch: 3/3... Step: 17860... Loss: 0.663820... Val Loss: 0.646952
Epoch: 3/3... Step: 17880... Loss: 0.733536... Val Loss: 0.648137
Epoch: 3/3... Step: 17900... Loss: 0.524619... Val Loss: 0.648071
Epoch: 3/3... Step: 17920... Loss: 0.591855... Val Loss: 0.648660
Epoch: 3/3... Step: 17940... Loss: 0.514012... Val Loss: 0.651123
Epoch: 3/3... Step: 17960... Loss: 0.718054... Val Loss: 0.647516
Epoch: 3/3... Step: 17980... Loss: 0.750166... Val Loss: 0.649719
Epoch: 3/3

In [242]:
torch.save(net.state_dict(), "model.pth")

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [288]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [t for t in tokens if t in vocab]
    # Convert words to ids
    tokens = [vocab[w] for w in tokens]
    
    tokens = np.array(tokens).reshape(1, -1)
    tokens = pad_features(tokens, 40)
        
    model.eval()
    tokens = torch.from_numpy(np.array(tokens)).view(1, -1)

    val_h = model.init_hidden(1)
    val_h = tuple([each.data.cuda() for each in val_h])
    output, _ = model(tokens.long().cuda(), val_h)


    pred = np.exp(output.detach().cpu().numpy())
    return pred

In [292]:
text = "Google is working on self driving cars, I'm bullish on $goog"
net.eval()
# model.to("cpu")
prediction = predict(text, net, vocab)[0]
print(prediction)

[4.5333325e-04 4.1924067e-02 1.5848929e-02 8.7575263e-01 6.6021137e-02]


D:\Anaconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

In [293]:
print(1.0 - np.max(prediction))

0.12424737215042114


Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [ ]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [ ]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

Using the `prediction` function, let's apply it to a stream of twits.

In [ ]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [ ]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.